#### 이번주 과제

- 네이버 증권 업종별 시세에서 종목코드를 하나하나 크롤링 ⇒ 그 종목코드를 10년치 일별 데이터 db와 연결해서 데이터를 가져오고 10년 이상 인지 확인하고, 합병이 되었는지 확인 ⇒ 그 중 5개를 뽑는 것 (시가총액, 10년, 코스닥 or 코스피인지) ⇒ 리스트에 넣음


- 크롤링으로 상장일, 시가총액, 코스피 or 코스닥 구분



### 4월2일 금요일 
업종별 시세 -> 식품 
식품 업종 기업들의 기업명/종목코드/시가총액/per/코스피/상장일 크롤링<br>
csv 파일로 저장 <br>
데이터프레임 완성 ✯★☆✧

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from glob import glob
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from tqdm import tqdm

In [2]:
# 크롬드라이버 셋팅 
driver = webdriver.Chrome('E:/big12/python-project/note/driver/chromedriver.exe')
# 네이버 증권 화면으로 이동 
driver.get('https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=38')
# 네이버 증권홈 -> 국내증시 -> 업종별시세:식품 페이지로 지정

In [3]:
# 기본 선택됨 : 거래량, 매수호가, 거래대금, 매도호가, 전일거래량
# 모두 선택 해제 후 다시 선택 -> 시가총액, per

# 거래량
georae_yang = driver.find_element_by_id('option1')
# 매수호가
maesoo_hoga = driver.find_element_by_id('option2')
# 거래대금
georae_daegum = driver.find_element_by_id('option3')
# 매도호가
maedo_hoga = driver.find_element_by_id('option8')
# 전일거래량 
jeon_il_geora = driver.find_element_by_id('option9')


# 선택해제 
georae_yang.click()
maesoo_hoga.click()
georae_daegum.click()
maedo_hoga.click()
jeon_il_geora.click()

# 시가총액
siga_chong = driver.find_element_by_id('option4') 
# per
per = driver.find_element_by_id('option6')

# 선택 
siga_chong.click()
per.click()

# 적용하기 클릭 
xpath = '''//*[@id="contentarea"]/div[3]/form/div/div/div/a[1]'''
apply = driver.find_element_by_xpath(xpath)
apply.click()

In [4]:
# 기업명, 시가총액, per 가져오기 

name_list = []
market_cap_list = []
per_list = []

xpath = '''//*[@id="contentarea"]/div[4]/table/tbody'''
table = driver.find_element_by_xpath(xpath).text

# 테이블 안의 이름 영역 
name_area = driver.find_elements_by_class_name('name_area')
# 테이블 안의 모든 숫자 -> td class="number" 
number_area = driver.find_elements_by_class_name('number')

# 기업명
name_list = [x.find_element_by_tag_name('a').text for x in name_area]

# 시가총액, per
for i in range(len(number_area)):
    # 3번째 인덱스인 시가총액
    if(i%5==3):
        market_cap_list.append((number_area[i].text).replace(',',''))
    # 4번째 인덱스인 per
    elif(i%5==4):
        per_list.append(number_area[i].text)
        
    
# 첫 번째 값이 이상하당 빼버리자 
market_cap_list = market_cap_list[1:]
per_list = per_list[1:]

In [5]:
# 기업 페이지 url
name_box = driver.find_elements_by_class_name('name_area')
company_page = [x.find_element_by_tag_name('a').get_attribute('href')  for x in name_box]

# 각 기업 페이지 접속 -> 종목코드/클래스/상장일 저장
# 84번 반복합니다 ..! 

code_list = []
class_list = []
reg_day_list = []

for i in tqdm(range(len(company_page))):
    driver.get(company_page[i])

    # 종목코드 
    code = driver.find_element_by_class_name('code').text
    # print('종목코드는: ', code)
    code_list.append(code)

    # 코스피,코스닥 클래스 
    try:
        driver.find_element_by_class_name('kospi')
        class_ = 'kospi'
    except:
        class_ = 'kosdaq'

    # print('코스피 or 코스닥? :', class_)
    class_list.append(class_)


    # 상장일 
    # url에 종목코드 넣어서 포맷 
    driver.get('https://navercomp.wisereport.co.kr/v2/company/c1020001.aspx?cmp_cd={code}&cn='.format(code=code))

    # 기업개요 클릭
    btn = driver.find_element_by_class_name('line-left')
    btn.click()

    reg_day_long = driver.find_element_by_css_selector('#cTB201 > tbody > tr:nth-child(3) > td.c2.txt').text
    x = reg_day_long.split()
    reg_day = x[-1].replace(')','')
    # print('상장일: ',reg_day)
    reg_day_list.append(reg_day)

# 종료
driver.close()



100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [01:09<00:00,  1.21it/s]


In [6]:
df = pd.DataFrame(
    {
         '기업명':name_list
        ,'시가총액':market_cap_list
        ,'per':per_list
        ,'종목코드':code_list
        ,'클래스':class_list
        ,'상장일':reg_day_list
    }
)

In [7]:
# 데이터프레임 합치기 ~~~~~~
 #lala = pd.concat([df,df2],axis=1, join='outer')

In [8]:
df

,기업명,시가총액,per,종목코드,클래스,상장일
0,대한제당우,201,13.19,001795,kospi,1968/12/27
1,대한제당,3148,14.93,001790,kospi,1968/12/27
2,신라에스지,704,-51.31,025870,kosdaq,1995/07/07
3,동서,33449,28.58,026960,kospi,2016/07/15
4,크라운해태홀딩스우,90,10.03,005745,kospi,1976/06/30
...,...,...,...,...,...,...
79,대상홀딩스,3712,4.34,084690,kospi,2005/08/17
80,한탑,481,-2.41,002680,kosdaq,1995/07/07
81,우리손에프앤지,1606,7.37,073560,kosdaq,2016/07/27
82,하림지주,8588,15.76,003380,kosdaq,2017/06/30


In [15]:
# csv로 만들어줘 ~~~~~~
pd.DataFrame(df).to_csv('./categories/손냥이의식품전체.csv')

In [10]:
# 데이터 전처리 

food = pd.read_csv('./categories/손냥이의식품전체.csv', encoding='utf-8')
food

,Unnamed: 0,기업명,시가총액,per,종목코드,클래스,상장일
0,0,대한제당우,201,13.19,001795,kospi,1968/12/27
1,1,대한제당,3148,14.93,001790,kospi,1968/12/27
2,2,신라에스지,704,-51.31,025870,kosdaq,1995/07/07
3,3,동서,33449,28.58,026960,kospi,2016/07/15
4,4,크라운해태홀딩스우,90,10.03,005745,kospi,1976/06/30
...,...,...,...,...,...,...,...
79,79,대상홀딩스,3712,4.34,084690,kospi,2005/08/17
80,80,한탑,481,-2.41,002680,kosdaq,1995/07/07
81,81,우리손에프앤지,1606,7.37,073560,kosdaq,2016/07/27
82,82,하림지주,8588,15.76,003380,kosdaq,2017/06/30


In [12]:
food.sort_values(by=['시가총액','상장일'],ascending=[False,True])

,Unnamed: 0,기업명,시가총액,per,종목코드,클래스,상장일
33,33,오리온,51397,19.21,271560,kospi,2017/07/07
23,23,롯데지주,35354,-17.98,004990,kospi,1974/02/16
3,3,동서,33449,28.58,026960,kospi,2016/07/15
27,27,오뚜기,20967,19.49,007310,kospi,1994/08/12
55,55,농심,17244,11.61,004370,kospi,1976/06/30
...,...,...,...,...,...,...,...
26,26,서울식품우,113,-724.29,004415,kospi,1973/12/28
4,4,크라운해태홀딩스우,90,10.03,005745,kospi,1976/06/30
10,10,크라운제과우,82,8.70,26490K,kospi,2017/04/11
42,42,대상홀딩스우,77,3.56,084695,kospi,2005/08/17


In [10]:
food=pd.read_csv('./캬옹.csv')

In [11]:
food

,Unnamed: 0,Unnamed: 0.1,기업명,시가총액,per,종목코드,클래스,상장일
0,60,60,삼양홀딩스,7871,7.97,70,kospi,1968/12/27
1,1,1,대한제당,3148,14.93,1790,kospi,1968/12/27
2,0,0,대한제당우,201,13.19,1795,kospi,1968/12/27
3,61,61,삼양홀딩스우,183,5.22,75,kospi,1968/12/27
4,34,34,대상,8801,7.31,1680,kospi,1970/04/01
